In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://thewaltdisneycompany.com/investor-relations/"

response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.text, 'html.parser')

pdf_links = soup.find_all('a', class_='pdf-file')

save_dir = "disney_reports"
os.makedirs(save_dir, exist_ok=True)

# Choose what to download: 'annual' or 'quarterly'
report_type = 'annual' 

for link in pdf_links:
    pdf_url = link.get('href')
    if pdf_url.endswith('.pdf'):
        link_text = link.get_text(strip=True).lower() 
        
        if (report_type == 'annual' and 'annual report' in link_text) or \
           (report_type == 'quarterly' and ('q1' in link_text or 'q2' in link_text or 'q3' in link_text or 'q4' in link_text or 'earnings report' in link_text)):
            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()
            
            file_name = pdf_url.split('/')[-1]
            save_path = os.path.join(save_dir, file_name)
            
            with open(save_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {file_name}")

print(f"All {report_type} have been downloaded.")


In [21]:
pip install PyMuPDF tabula-py pandas


  Obtaining dependency information for tabula-py from https://files.pythonhosted.org/packages/d5/77/b34088cbb55ba59e1cc6512ab2ff3b7679102b7f7577982a96cbdcddb90c/tabula_py-2.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 1.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import pandas as pd
from tabula import read_pdf, convert_into

def extract_tables(pdf_path, output_dir):
    output_path = os.path.join(output_dir, os.path.basename(pdf_path).replace('.pdf', '.csv'))
    try:
        convert_into(pdf_path, output_path, output_format="csv", pages="all")
        print(f"Tables extracted from {os.path.basename(pdf_path)} and saved as {output_path}")
    except Exception as e:
        print(f"Failed to extract tables from {os.path.basename(pdf_path)}: {e}")

def process_pdfs(pdf_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(pdf_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            extract_tables(pdf_path, output_dir)

pdf_directory = 'disney_reports'
output_directory = 'extracted_reports'
process_pdfs(pdf_directory, output_directory)


Got stderr: Mar 01, 2024 2:23:54 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Mar 01, 2024 2:23:54 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
OpenJDK 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
OpenJDK 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=
Mar 01, 2024 2:23:55 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Mar 01, 2024 2:24:10 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Mar 01, 2024 2:24:12 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Mar 01, 2024 2:24:12 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
Mar 01, 2024 2:24:12 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode



Tables extracted from 2020-Annual-Report.pdf and saved as extracted_reports/2020-Annual-Report.csv


Got stderr: OpenJDK 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
OpenJDK 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=
Mar 01, 2024 2:24:16 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Mar 01, 2024 2:24:16 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed
Mar 01, 2024 2:24:16 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed



Tables extracted from 2014-Annual-Report.pdf and saved as extracted_reports/2014-Annual-Report.csv


KeyboardInterrupt: 